In [16]:
import json
import os
!mkdir /home/aistudio/data/data274453/fusai-2024-cti-bigmodel-retrieval-data
!tar  -zxvf /home/aistudio/data/data274453/fusai-2024-cti-bigmodel-retrieval-data.tar.gz -C /home/aistudio/data/data274453/fusai-2024-cti-bigmodel-retrieval-data
os.chdir("/home/aistudio/data/data274453/fusai-2024-cti-bigmodel-retrieval-data")
!ls

fusai-2024-cti-bigmodel-retrieval-data/
fusai-2024-cti-bigmodel-retrieval-data/train_data.csv
fusai-2024-cti-bigmodel-retrieval-data/ad_landingpage.txt
fusai-2024-cti-bigmodel-retrieval-data/ad_core_term.txt
fusai-2024-cti-bigmodel-retrieval-data/dev_data.csv
ad_core_term.txt    dev_data.csv	 fusai_train_data.json
ad_landingpage.txt  fusai_dev_data.json  train_data.csv


In [11]:
#将训练和验证数据写入为json格式
with open('dev_data.csv', 'r') as f:
    with open('fusai_dev_data.json', 'w', buffering=1024*1024) as w:
        lines = f.readlines()
        for line in lines:
            src, tgt, click = line.split('\t')
            data = {"src":src, "tgt":tgt, "click":click}
            w.write(json.dumps(data, ensure_ascii=False) + "\n")
with open('train_data.csv', 'r') as f:
    with open('fusai_train_data.json', 'w', buffering=1024*1024) as w:
        lines = f.readlines()
        for line in lines:
            src, tgt, click = line.split('\t')
            data = {"src":src, "tgt":tgt, "click":click}
            w.write(json.dumps(data, ensure_ascii=False) + "\n")         

In [19]:
#统计落地页数据信息
import numpy as np
with open('ad_landingpage.txt', 'r', buffering=1024*1024) as f:
    lines = f.readlines()
    a = [len(line.split("<block>")) for line in lines]
    cnt = 0
    total_len = 0
    for line in lines:
        src = line.split('\t')[1]
        for i in src.split("<block>"):
            cnt += 1
            total_len += len(i)
    print(total_len / cnt)
    print(np.mean(a))
    print(len(lines))
    print(sum(a))

11.567208496080635
50.74696851851643
888427
45084977


In [16]:
#统计核心词数据信息
import numpy as np
with open('ad_core_term.txt', 'r', buffering=1024*1024) as f:
    lines = f.readlines()
    a = [len(line.split("<sep>")) for line in lines]
    print(np.mean(a))
    print(len(lines))
    print(sum(a))

2.776372172390078
888427
2466604


In [ ]:
#清理ollama生成的数据
import jieba
import json
from tqdm import *
with open("./fusai_train_data.json", "r") as f:
    lines = f.readlines()
    data = [json.loads(line) for line in lines]
d_key = dict()
for d in data:
#清洗思路为：要求每个广告词的长度在4-20之间，并且包含训练数据中使用jieba切分后的任意一个词即可
    keywords = set(jieba.cut(d['src']))
    d_key[int(d['tgt'])] = keywords

#此处为未清洗前的数据
#可以使用/home/aistudio/work/scripts/ollama/test_ollama_*得到
with open("/home/aistudio/work/scripts/ollama/train_indexing_qwen_merge_cleaned.json", "r") as f:
    lines = f.readlines()
    to_mine = [json.loads(line) for line in lines]

def contains_keyword(s, keywords):
    return any(keyword in s for keyword in keywords)

#清洗数据：
mined_data = []
for item in to_mine:
    split_data = item['src'].split(";")
    print(split_data)
    if int(item['tgt']) not in d_key:
        print(  f"不存在{item['tgt']}")
        mined_data.extend([{'tgt': item['tgt'], 'src': s} for s in split_data])
        continue
    print(d_key[int(item['tgt'])])
    
    filtered_data = [s for s in split_data if 4 < len(s) < 20 and contains_keyword(s, d_key[int(item['tgt'])])]
    filtered_data = set(filtered_data)
    # Add the data to the mined data
    mined_data.extend([{'tgt': item['tgt'], 'src': s} for s in filtered_data])

# Print the mined data
with open('/home/aistudio/work/data/fusai_cleaned_ollama_1.json', "w", buffering=1024*1024) as w:
    for item in tqdm(mined_data):
        # print(item)
        w.write(json.dumps(item,ensure_ascii=False) +"\n")

In [14]:
#合并数据
with open("/home/aistudio/work/data/fusai_cleaned_ollama_1.json", "r") as f:
    with open("./fusai_train_data.json", "r") as f1:
       with open("/home/aistudio/work/data/fusai_query.json", "w", buffering=1024*1024) as w:
        for line in f:
            w.write(line)
        for line in f1:
            w.write(line)

In [12]:
#统计训练数据每个tgt对应src和click的数量
d_query = dict()
d_max_click = dict()
with open('./fusai_train_data.json', 'r') as w:
    for line in w:
        data = json.loads(line)
        src, tgt, click = data['src'], int(data['tgt']), float(data['click'])
        if tgt not in d_query:
            d_query[tgt] = []
        d_query[tgt].append([src,click])
        if tgt not in d_max_click:
            d_max_click[tgt] = click
        else:
            d_max_click[tgt] = max(d_max_click[tgt],click)
# click的分位数 (25%, 50%, 90%): [1. 1. 4.]
# 每个tgt包含的[src, click]对个数的分位数 (10%, 50%, 75%): [4.4 6.  8.5]
use_tgts = []
for tgt in range(1,888427 + 1):
    if tgt not in d_query or d_max_click[tgt] >= 4.0 or len(d_query) < 5:
        use_tgts.append(tgt)
len(use_tgts)


437851

In [13]:
import numpy as np
# 提取所有click的值
clicks = [click for src_click_pairs in d_query.values() for src, click in src_click_pairs]
# print(clicks)
# 计算click的中位数
# 计算click的25%、50%（中位数）和75%分位数
click_percentiles = np.percentile(clicks, [25, 50, 90])

# 计算每个tgt包含的[src, click]对个数
tgt_counts = [len(src_click_pairs) for src_click_pairs in data.values()]

# 计算每个tgt包含的[src, click]对个数的25%、50%（中位数）和75%分位数
tgt_count_percentiles = np.percentile(tgt_counts, [10, 50, 75])
print(np.max(clicks))
print(np.mean(clicks))
print("click的分位数 (25%, 50%, 75%):", click_percentiles)
print("每个tgt包含的[src, click]对个数的分位数 (25%, 50%, 75%):", tgt_count_percentiles)

50.0
2.5756564
click的分位数 (25%, 50%, 75%): [1. 1. 4.]
每个tgt包含的[src, click]对个数的分位数 (25%, 50%, 75%): [4.4 6.  8.5]


In [1]:
#对ollama生成的每一项进行进一步清洗
#注意：该步骤需要迭代重复两次
import json
with open("/home/aistudio/work/data/fusai_cleaned_ollama_1.json", "r") as f:
    with open("/home/aistudio/work/data/fusai_cleaned_ollama_1_tmp.json", "w") as w:
        for line in f:
            data = json.loads(line)
            src = data['src']
            if "；" in src:
                srcs = src.split("；")
                for s in srcs:
                    src = s
                    if "\n" in src:
                        src = src.replace("\n","")
                    if "1. " in src:
                        src = src.replace("1. ","")
                    if "2. " in src:
                        src = src.replace("2. ","")
                    if "3. " in src:
                        src = src.replace("3. ","")
                    src = src.strip()
                    if len(src) == 0 or len(src) > 20:
                        continue
                    w.write(json.dumps({"src":s,"tgt":data["tgt"]}, ensure_ascii=False) + "\n")
                continue
            if "\n" in src:
                src = src.replace("\n","")
            if "|" in src:
                src = src.replace("|","")
            if "1. " in src:
                src = src.replace("1. ","")
            if "2. " in src:
                src = src.replace("2. ","")
            if "3. " in src:
                src = src.replace("3. ","")
            if "、" in src:
                src = src.replace("、","、")
            if "\n" in src:
                src = src.replace("\n","")
            if '\"' in src:
                src = src.replace('\"',"")
            if len(src.split(" ")) == 3:
                continue
            src = src.strip()
            if len(src) == 0 or len(src) > 20 or len(src) < 4:
                continue
            else:
                w.write(json.dumps({"src":src,"tgt":data["tgt"]}, ensure_ascii=False) + "\n")

In [ ]:
#合并ollama数据
with open("/home/aistudio/work/data/fusai_cleaned_ollama_all.json", "w", buffering=1024 * 1024) as w:
    with open("/home/aistudio/work/scripts/ollama/test_ollama_group_gemma2_data.json", "r") as f:
        for line in f:
            w.write(line)
    with open("/home/aistudio/work/scripts/ollama/test_ollama_group_all_data.json", "r") as f:
        for line in f:
            w.write(line)
    with open("/home/aistudio/work/scripts/ollama/test_ollama_group_qwen1.8_data.json", "r") as f:
        for line in f:
            w.write(line)
    with open("/home/aistudio/work/scripts/ollama/test_ollama_group_data.json", "r") as f:
        for line in f:
            w.write(line)
    with open("/home/aistudio/work/data/fusai_cleaned_ollama_1.json", "r") as f:
        for line in f:
            w.write(line)
    